## **Project 1: Financial Assistant**

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from openai.types.beta import Assistant
import yfinance as yf
from openai.types.beta.threads import Message
import time

In [3]:
import warnings

# Ignore only DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [5]:
_

True

In [6]:
# Get the key
api_key = os.getenv("OPENAI_API_KEY")

# Check and print whether it's loaded
if api_key:
    print("🔐 OpenAI Key Loaded:", bool(api_key))  # True or False
if not api_key:
    raise EnvironmentError("❌ OPENAI_API_KEY not found in .env file!")

🔐 OpenAI Key Loaded: True


In [7]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

#### **Defining custom function**

In [8]:
def get_stock_summary(ticker: str):
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        "name": info.get("shortName"),
        "price": info.get("regularMarketPrice"),
        "market_cap": info.get("marketCap"),
        "pe_ratio": info.get("trailingPE"),
        "summary": info.get("longBusinessSummary", ""),
        "location": info.get("city", "") + ", " + info.get("state", "")
    }

def get_crypto_summary(symbol: str):
    try:
        crypto = yf.Ticker(symbol)
        info = crypto.info
        return {
            "name": info.get("shortName", symbol),
            "price": info.get("regularMarketPrice", "N/A"),
            "market_cap": info.get("marketCap", "N/A"),
            "summary": info.get("longBusinessSummary", "No description available.")
        }
    except Exception as e:
        return {"error": str(e)}


In [9]:
available_functions = {
    "get_stock_summary": get_stock_summary,
    "get_crypto_summary": get_crypto_summary
}

## **Tools Schemas**

In [10]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_summary",
            "description": "Get a stock's market summary by ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Stock symbol like AAPL, TSLA"
                    }
                },
                "required": ["ticker"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_crypto_summary",
            "description": "Get a cryptocurrency summary by symbol like BTC-USD",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "Crypto symbol, e.g., BTC-USD, ETH-USD"
                    }
                },
                "required": ["symbol"]
            }
        }
    }
]


## **Create Assistants**

In [11]:
assistant : Assistant = client.beta.assistants.create(
    name="AI Financial Analyst",
    instructions="You are a professional financial analyst. Use functions as needed to analyze assets.",
    tools=tools,
    model="gpt-3.5-turbo-1106"
)

## **Create threads**

In [12]:
from openai.types.beta.thread import Thread

thread : Thread = client.beta.threads.create()

## **Helper Functions**

In [13]:
def add_message_to_thread(client, thread_id, content: str):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )

def start_run(client, thread_id, assistant_id):
    return client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )



## **Run and Polling**

In [14]:
def run_and_poll(client, thread_id, run_id, available_functions):
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Run Status: {run_status.status}")

        if run_status.status == "requires_action":
            print("Assistant requires action (tool calls)...")
            tool_calls = run_status.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                print(f"Tool Call: {function_name}({function_args})")

                if function_name in available_functions:
                    try:
                        result = available_functions[function_name](**function_args)
                        if not isinstance(result, str):
                            result = json.dumps(result)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": result
                        })
                    except Exception as e:
                        print(f"Error in {function_name}: {e}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": json.dumps({"error": str(e)})
                        })

            print("Submitting tool outputs...")
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread_id,
                run_id=run_id,
                tool_outputs=tool_outputs
            )

        elif run_status.status == "completed":
            print("Run completed. Fetching final assistant message...\n")

            # Filter only messages created in this specific run
            messages = client.beta.threads.messages.list(thread_id=thread_id)
            for message in reversed(messages.data):
                if message.role == "assistant" and message.run_id == run_id:
                    reply = message.content[0].text.value
                    print("Assistant Reply:\n", reply)
                    return reply
            return None

        elif run_status.status in ["in_progress", "queued"]:
            time.sleep(2)

        elif run_status.status == "failed":
            print("Run failed.")
            return None

        else:
            print(f"Unexpected status: {run_status.status}")
            return None


In [15]:
# ✅ Send message + run assistant + handle tools
def send_and_run_message(client, thread_id, assistant_id, content, available_functions):
    print(f"\n💬 New User Message: {content}")

    # Step 1: Add message
    add_message_to_thread(client, thread_id, content)

    # Step 2: Start run
    run = start_run(client, thread_id, assistant_id)

    # Step 3: Poll and return final reply
    return run_and_poll(client, thread_id, run.id, available_functions)

In [16]:
tsla_performance = send_and_run_message(
    client=client,
    thread_id=thread.id,
    assistant_id=assistant.id,
    content="Analyze TSLA performance.",
    available_functions=available_functions
)


💬 New User Message: Analyze TSLA performance.
Run Status: queued
Run Status: requires_action
Assistant requires action (tool calls)...
Tool Call: get_stock_summary({'ticker': 'TSLA'})
Submitting tool outputs...
Run Status: queued
Run Status: in_progress
Run Status: completed
Run completed. Fetching final assistant message...

Assistant Reply:
 Tesla, Inc. (TSLA) is currently priced at $329.65 with a market capitalization of $1,061,789,433,856. The company focuses on designing, developing, manufacturing, leasing, and selling electric vehicles and energy generation and storage systems. They operate in two segments: Automotive and Energy Generation and Storage. The Automotive segment offers electric vehicles, automotive regulatory credits, and various after-sales services, while the Energy Generation and Storage segment focuses on solar energy generation, energy storage products, and related services.

The stock's price-to-earnings (P/E) ratio is 188.37, indicating a relatively high valu

In [17]:
BTC_performance = send_and_run_message(
    client=client,
    thread_id=thread.id,
    assistant_id=assistant.id,
    content="Analyze BTC performance.",
    available_functions=available_functions
)



💬 New User Message: Analyze BTC performance.
Run Status: queued
Run Status: requires_action
Assistant requires action (tool calls)...
Tool Call: get_crypto_summary({'symbol': 'BTC-USD'})
Submitting tool outputs...
Run Status: queued
Run Status: completed
Run completed. Fetching final assistant message...

Assistant Reply:
 Bitcoin (BTC) is currently priced at $118,327.87 with a market capitalization of $2,354,173,968,384. Unfortunately, there is no further description available for this asset.
